In [13]:
import tensorflow
import tensorflow as tf

In [71]:
features = 110 #784
classes = 1 #10

a_0 = tf.placeholder(tf.float32, [None, features])
y = tf.placeholder(tf.float32, [None, classes])

middle = 20 #int(features / 3)
w_1 = tf.Variable(tf.truncated_normal([features, middle]))
b_1 = tf.Variable(tf.truncated_normal([1, middle]))
w_2 = tf.Variable(tf.truncated_normal([middle, classes]))
b_2 = tf.Variable(tf.truncated_normal([1, classes]))

In [72]:
def sigma(x):
    return tf.div(tf.constant(1.0),
                  tf.add(tf.constant(1.0), tf.exp(tf.neg(x))))

### Forward Propagation

In [73]:
z_1 = tf.add(tf.matmul(a_0, w_1), b_1)
a_1 = sigma(z_1)
z_2 = tf.add(tf.matmul(a_1, w_2), b_2)
a_2 = sigma(z_2)

### Error

In [74]:
diff = tf.sub(a_2, y)

### Sigmoid Derivative

In [75]:
def sigmaprime(x):
    return tf.mul(sigma(x), tf.sub(tf.constant(1.0), sigma(x)))

### Back Propagation

In [76]:
d_z_2 = tf.mul(diff, sigmaprime(z_2))
d_b_2 = d_z_2
d_w_2 = tf.matmul(tf.transpose(a_1), d_z_2)

d_a_1 = tf.matmul(d_z_2, tf.transpose(w_2))
d_z_1 = tf.mul(d_a_1, sigmaprime(z_1))
d_b_1 = d_z_1
d_w_1 = tf.matmul(tf.transpose(a_0), d_z_1)

In [77]:
eta = tf.constant(0.5)
step = [
    tf.assign(w_1,
            tf.sub(w_1, tf.mul(eta, d_w_1)))
  , tf.assign(b_1,
            tf.sub(b_1, tf.mul(eta,
                               tf.reduce_mean(d_b_1, reduction_indices=[0]))))
  , tf.assign(w_2,
            tf.sub(w_2, tf.mul(eta, d_w_2)))
  , tf.assign(b_2,
            tf.sub(b_2, tf.mul(eta,
                               tf.reduce_mean(d_b_2, reduction_indices=[0]))))
]

### 2 Sigma Learning

In [78]:
acct_mat = tf.equal(tf.argmax(a_2, 1), tf.argmax(y, 1))
acct_res = tf.reduce_sum(tf.cast(acct_mat, tf.float32))

init = tf.global_variables_initializer()

In [80]:
import numpy as np
import kagglegym

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(0, 2):
        env = kagglegym.make()
        observation = env.reset()

        done = False
        while not done:
            target = observation.target
            timestamp = observation.features["timestamp"][0]

            batch_xs = observation.features.fillna(0)
            target["y"] = sess.run(a_2, feed_dict = {a_0: batch_xs})
            #print(target["y"])

            observation, reward, done, info = env.step(target)

            batch_ys = np.vstack([reward] * classes)
            sess.run(step, feed_dict = {a_0: batch_xs,
                                        y : batch_ys})

            if timestamp % 100 == 0:
                print("Epoch #{0}, Timestamp #{1}, Reward {2}".format(epoch, timestamp, reward))
                #break

        
info

Epoch #0, Timestamp #1000, Reward -0.009768245129833153
Epoch #0, Timestamp #1100, Reward -0.0784853453190204
Epoch #0, Timestamp #1200, Reward -0.06722142736796209
Epoch #0, Timestamp #1300, Reward -0.03831479699670408
Epoch #0, Timestamp #1400, Reward -0.16841577420002168
Epoch #0, Timestamp #1500, Reward -0.16337528218121475
Epoch #0, Timestamp #1600, Reward -0.16900778147536097
Epoch #0, Timestamp #1700, Reward -0.025669278916603158
Epoch #0, Timestamp #1800, Reward -0.054221471311693206
Epoch #1, Timestamp #1000, Reward -0.009768245129833153
Epoch #1, Timestamp #1100, Reward -0.0784853453190204
Epoch #1, Timestamp #1200, Reward -0.06722142736796209
Epoch #1, Timestamp #1300, Reward -0.03831479699670408
Epoch #1, Timestamp #1400, Reward -0.16841577420002168
Epoch #1, Timestamp #1500, Reward -0.16337528218121475
Epoch #1, Timestamp #1600, Reward -0.16900778147536097
Epoch #1, Timestamp #1700, Reward -0.025669278916603158
Epoch #1, Timestamp #1800, Reward -0.054221471311693206


{'public_score': -0.0083462397338969572}

### MNIST Learning

In [43]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [51]:
acct_mat = tf.equal(tf.argmax(a_2, 1), tf.argmax(y, 1))
acct_res = tf.reduce_sum(tf.cast(acct_mat, tf.float32))

init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

for i in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(10)
    sess.run(step, feed_dict = {a_0: batch_xs,
                                y : batch_ys})
    if i % 1000 == 0:
        print(len(batch_xs), len(batch_xs[0]))
        res = sess.run(acct_res, feed_dict =
                       {a_0: mnist.test.images[:1000],
                        y : mnist.test.labels[:1000]})
        print(res)

10 784
124.0
10 784
743.0
10 784
783.0
10 784
805.0
10 784
798.0
10 784
818.0
10 784
809.0
10 784
816.0
10 784
833.0
10 784
914.0
